## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**3126. Server Utilization Time (Medium)**

**Table: Servers**

| Column Name    | Type     |
|----------------|----------|
| server_id      | int      |
| status_time    | datetime |
| session_status | enum     |

(server_id, status_time, session_status) is the primary key (combination of columns with unique values) for this table.
session_status is an ENUM (category) type of ('start', 'stop').
Each row of this table contains server_id, status_time, and session_status.

**Write a solution to find the total time when servers were running. The output should be rounded down to the nearest number of full days.**

Return the result table in any order.

The result format is in the following example.

**Example:**

**Input:**

**Servers table:**

| server_id | status_time         | session_status |
|-----------|---------------------|----------------|
| 3         | 2023-11-04 16:29:47 | start          |
| 3         | 2023-11-05 01:49:47 | stop           |
| 3         | 2023-11-25 01:37:08 | start          |
| 3         | 2023-11-25 03:50:08 | stop           |
| 1         | 2023-11-13 03:05:31 | start          |
| 1         | 2023-11-13 11:10:31 | stop           |
| 4         | 2023-11-29 15:11:17 | start          |
| 4         | 2023-11-29 15:42:17 | stop           |
| 4         | 2023-11-20 00:31:44 | start          |
| 4         | 2023-11-20 07:03:44 | stop           |
| 1         | 2023-11-20 00:27:11 | start          |
| 1         | 2023-11-20 01:41:11 | stop           |
| 3         | 2023-11-04 23:16:48 | start          |
| 3         | 2023-11-05 01:15:48 | stop           |
| 4         | 2023-11-30 15:09:18 | start          |
| 4         | 2023-11-30 20:48:18 | stop           |
| 4         | 2023-11-25 21:09:06 | start          |
| 4         | 2023-11-26 04:58:06 | stop           |
| 5         | 2023-11-16 19:42:22 | start          |
| 5         | 2023-11-16 21:08:22 | stop           |

**Output:**

| total_uptime_days |
|-------------------|
| 1                 |

**Explanation:**
- For server ID 3:
- From 2023-11-04 16:29:47 to 2023-11-05 01:49:47: ~9.3 hours
- From 2023-11-25 01:37:08 to 2023-11-25 03:50:08: ~2.2 hours
- From 2023-11-04 23:16:48 to 2023-11-05 01:15:48: ~1.98 hours

Total for server 3: ~13.48 hours
- For server ID 1:
- From 2023-11-13 03:05:31 to 2023-11-13 11:10:31: ~8 hours
- From 2023-11-20 00:27:11 to 2023-11-20 01:41:11: ~1.23 hours

Total for server 1: ~9.23 hours
- For server ID 4:
- From 2023-11-29 15:11:17 to 2023-11-29 15:42:17: ~0.52 hours
- From 2023-11-20 00:31:44 to 2023-11-20 07:03:44: ~6.53 hours
- From 2023-11-30 15:09:18 to 2023-11-30 20:48:18: ~5.65 hours
- From 2023-11-25 21:09:06 to 2023-11-26 04:58:06: ~7.82 hours

Total for server 4: ~20.52 hours
- For server ID 5:
- From 2023-11-16 19:42:22 to 2023-11-16 21:08:22: ~1.43 hours

Total for server 5: ~1.43 hours

The accumulated runtime for all servers totals approximately 44.46 hours, equivalent to one full day plus some additional hours. However, since we consider only full days, the final output is rounded to 1 full day.

In [0]:
servers_data_3126 = [
    (3, "2023-11-04 16:29:47", "start"),
    (3, "2023-11-05 01:49:47", "stop"),
    (3, "2023-11-25 01:37:08", "start"),
    (3, "2023-11-25 03:50:08", "stop"),
    (1, "2023-11-13 03:05:31", "start"),
    (1, "2023-11-13 11:10:31", "stop"),
    (4, "2023-11-29 15:11:17", "start"),
    (4, "2023-11-29 15:42:17", "stop"),
    (4, "2023-11-20 00:31:44", "start"),
    (4, "2023-11-20 07:03:44", "stop"),
    (1, "2023-11-20 00:27:11", "start"),
    (1, "2023-11-20 01:41:11", "stop"),
    (3, "2023-11-04 23:16:48", "start"),
    (3, "2023-11-05 01:15:48", "stop"),
    (4, "2023-11-30 15:09:18", "start"),
    (4, "2023-11-30 20:48:18", "stop"),
    (4, "2023-11-25 21:09:06", "start"),
    (4, "2023-11-26 04:58:06", "stop"),
    (5, "2023-11-16 19:42:22", "start"),
    (5, "2023-11-16 21:08:22", "stop"),
]

servers_columns_3126 = ["server_id", "status_time", "session_status"]
servers_df_3126 = spark.createDataFrame(servers_data_3126, servers_columns_3126)
servers_df_3126.show()


+---------+-------------------+--------------+
|server_id|        status_time|session_status|
+---------+-------------------+--------------+
|        3|2023-11-04 16:29:47|         start|
|        3|2023-11-05 01:49:47|          stop|
|        3|2023-11-25 01:37:08|         start|
|        3|2023-11-25 03:50:08|          stop|
|        1|2023-11-13 03:05:31|         start|
|        1|2023-11-13 11:10:31|          stop|
|        4|2023-11-29 15:11:17|         start|
|        4|2023-11-29 15:42:17|          stop|
|        4|2023-11-20 00:31:44|         start|
|        4|2023-11-20 07:03:44|          stop|
|        1|2023-11-20 00:27:11|         start|
|        1|2023-11-20 01:41:11|          stop|
|        3|2023-11-04 23:16:48|         start|
|        3|2023-11-05 01:15:48|          stop|
|        4|2023-11-30 15:09:18|         start|
|        4|2023-11-30 20:48:18|          stop|
|        4|2023-11-25 21:09:06|         start|
|        4|2023-11-26 04:58:06|          stop|
|        5|20

In [0]:
servers_df_3126 = servers_df_3126\
                    .withColumn("status_time", col("status_time").cast("timestamp"))

In [0]:
window_spec = Window.partitionBy("server_id").orderBy("status_time")

In [0]:
servers_with_lag_3126 = servers_df_3126\
                            .withColumn( "prev_status", lag("session_status").over(window_spec))\
                                .withColumn( "prev_time", lag("status_time").over(window_spec))

In [0]:
durations_3126 = servers_with_lag_3126\
                        .filter( (col("session_status") == "stop") & (col("prev_status") == "start"))\
                            .withColumn( "session_seconds", unix_timestamp("status_time") - unix_timestamp("prev_time"))

In [0]:
total_seconds_3126 = durations_3126\
                        .agg(sum("session_seconds").alias("total_seconds")).collect()[0]["total_seconds"]

In [0]:
total_days_3126 = int(total_seconds_3126 // (24 * 3600))

In [0]:
result = spark.createDataFrame([(total_days_3126,)], ["total_uptime_days"])
result.display()


total_uptime_days
1
